# Initialization

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline
from tqdm import tqdm
import joblib

In [2]:
import qgrid
qgrid.enable()
qgrid.set_grid_option('forceFitColumns',False)

In [3]:
# Read in data dictionary
HMAHCC_DataDictionary = pd.read_excel('HMAHCC_DataDictionary.xlsx','Event Attributes')

In [4]:
# Read in test dataset
HMAHCC_COMP = pd.read_csv('HMAHCC_HOLDOUT.csv',low_memory=False,na_values='NULL')

In [5]:
# Rename the columns to match with training data
HMAHCC_COMP.columns = ['id', 'event_descr', 'event_attr1', 'event_attr2', 'event_attr3',
        'event_attr4', 'event_attr5', 'event_attr6', 'event_attr7',
        'event_attr8', 'event_attr9', 'event_attr10',
        'PAY_DAY_SUPPLY_CNT', 'PAYABLE_QTY', 'MME', 'DRUG_TYPE', 'Specialty',
        'Specialty2', 'Specialty3', 'Days']

# Preprocess Attributes

In [6]:
def preprocess_attr(row):
    '''
    Proprocess the HMAHCC data to make it have correct attribute names
    
    :param row: a row from data dictionary file
    :return: subset of HMAHCC data with the same event_descr in row
    '''
    # Get event description of the row
    event_description = row["Event Description"]
    # Filter the HMAHCC data to the event description in row
    COMP_filtered = HMAHCC_COMP.query('event_descr == @event_description')
    # Select all columns has 'event_attr' in column names
    COMP_attr = COMP_filtered.filter(like='event_attr')
    # Rename thoese event_attr columns with the attribute names in row
    COMP_attr.columns = row.filter(like='Attribute')
    # From attr columns, select columns whose names are not null(not attribute in dictionary)
    COMP_attr_not_null = COMP_attr.loc[:, COMP_attr.columns.notnull()]
    # Get the the rest of the columns other than attr
    COMP_rest = COMP_filtered.iloc[:,~COMP_filtered.columns.str.contains('event_attr')]
    # Combine non-attr and attr columns together
    COMP = pd.concat([COMP_rest,COMP_attr_not_null],axis=1)
    return COMP

In [7]:
# Apply preprocess_attr to each row in data dictionary
results = [preprocess_attr(row) for n,row in tqdm(HMAHCC_DataDictionary.iterrows())]

16it [00:01,  9.93it/s]


In [8]:
# Concat processed HMAHCC data with different event_descr together
HMAHCC_COMP_processed = pd.concat(results, axis=0, ignore_index=True,sort=False)\
    .astype({'Member Responsible Amount': 'float'})

In [9]:
# Replace space in the column names with _
HMAHCC_COMP_processed.columns = HMAHCC_COMP_processed.columns.str.replace(' ', '_')

# LTOT Identification

## Filtering

In [10]:
# Find all rejected and reversal rx, group by unique identifiers of an rx, count reversal times
reversal = HMAHCC_COMP_processed\
    .query("event_descr=='RX Claim - Rejected'")\
    .query('Status_Code=="REVERSAL"')\
    .groupby(['id','Days','Brand_Name','Generic_Name','Member_Responsible_Amount'])\
    .size().to_frame('reversal_times')

In [11]:
# Join reversal rx to paid scripts, find the paid scripts that get rejected, also count times the patient paid
# This is for avoiding the situation the first paid script get rejected, but the second one goes through
# reversal_multiplier == 1 would be the paid scripts that get rejected
paid_reversal = HMAHCC_COMP_processed\
    .query("event_descr=='RX Claim - Paid'")\
    .merge(reversal,on=['id','Days','Brand_Name','Generic_Name','Member_Responsible_Amount'],
           how='inner',suffixes=['','_right'])\
    .groupby(['id','event_descr','Days','Brand_Name','Generic_Name','Member_Responsible_Amount','reversal_times'])\
    .size().to_frame('paid_times').reset_index()\
    .assign(actual_times=lambda df:df.paid_times - df.reversal_times)\
    .assign(reversal_multiplier=lambda df:df.actual_times/df.paid_times)\
    .assign(reversal_multiplier=lambda df: np.where(df.reversal_multiplier<0,0,df.reversal_multiplier))\
    .drop(columns=['reversal_times','actual_times','paid_times','actual_times'])

In [12]:
# Join reversal adjustment to HMAHCC data
HMAHCC_COMP_processed_reversed = HMAHCC_COMP_processed\
    .merge(paid_reversal,
           on=['id','event_descr','Days','Brand_Name','Generic_Name','Member_Responsible_Amount'],
           how='left')

# Feature Engineering

In [13]:
# Read in preprocessed HMAHCC data
HMAHCC = HMAHCC_COMP_processed_reversed

In [14]:
# Get state from location column, filter data to Days<=0, remove rejected scripts, fill na in descriptive columns
HMAHCC = HMAHCC\
    .assign(state = lambda df: df.Location.fillna('').str.split(', ').apply(lambda x:x[-1]))\
    .query('reversal_multiplier!=0')\
    .fillna({'Diagnosis':'','Place_of_Treatment':'','Drug_Group_Description':'',
             'DRUG_TYPE':''})

## Inbound Call by Mbr

### State

In [15]:
# Get state for each patient, for patients who has multiple states, get the one with max count
# state = HMAHCC\
#     .query('event_descr == "Inbound Call by Mbr"')\
#     .groupby(['id','state'])\
#     .size().to_frame('state_count').reset_index()\
#     .set_index('id')\
#     .assign(state_max=lambda df:df.groupby('id').state_count.max())\
#     .query('state_count == state_max')\
#     [['state']].reset_index()\
#     .sort_values(['id','state'],ascending=[True,False])\
#     .drop_duplicates(subset='id')\
#     .set_index('id').pipe(pd.get_dummies)

### Frequency

In [16]:
# Calculate the frequency of inbound call by member
freq_inbound_call_by_mbr = HMAHCC\
    .query('event_descr == "Inbound Call by Mbr"')\
    .groupby('id')\
    .size().to_frame('freq_inbound_call_by_mbr')

### Recency

In [17]:
# Calculate the recency of inbound call by member
rec_inbound_call_by_mbr = HMAHCC\
    .query('event_descr == "Inbound Call by Mbr"')\
    .groupby('id')\
    .Days.max().abs().to_frame('rec_inbound_call_by_mbr')

## Inbound Call by Other

### Frequency

In [18]:
# Calculate the frequency of inbound call by other
freq_inbound_call_by_other = HMAHCC\
    .query('event_descr == "Inbound Call by Other"')\
    .groupby('id')\
    .size().to_frame('freq_inbound_call_by_other')

### Recency

In [19]:
# Calculate the recency of inbound call by other
rec_inbound_call_by_other = HMAHCC\
    .query('event_descr == "Inbound Call by Other"')\
    .groupby('id')\
    .Days.max().abs().to_frame('rec_inbound_call_by_other')

## Inbound Call by Prov

### Frequency

In [20]:
# Calculate the frequency of inbound call by provider
freq_inbound_call_by_prov = HMAHCC\
    .query('event_descr == "Inbound Call by Prov"')\
    .groupby('id')\
    .size().to_frame('freq_inbound_call_by_prov')

### Recency

In [21]:
# Calculate the recency of inbound call by provider
rec_inbound_call_by_prov = HMAHCC\
    .query('event_descr == "Inbound Call by Prov"')\
    .groupby('id')\
    .Days.max().abs().to_frame('rec_inbound_call_by_prov')

## Fully Paid Claim

### Diagnosis Frequency

In [22]:
# Identify top 5 diagnosis from diagnosis column, and the rest assigned to others
fully_paid_claim_diag = HMAHCC\
    .query('event_descr == "Fully Paid Claim"')\
    .assign(diag_hypertension = lambda df:df.Diagnosis.str.contains('HYPERTENSION'))\
    .assign(diag_diabete = lambda df:df.Diagnosis.str.contains('DIABETE'))\
    .assign(diag_CAD = lambda df:df.Diagnosis.str.contains('CORONARY'))\
    .assign(diag_CPD = lambda df:df.Diagnosis.str.contains('PULMONARY'))\
    .assign(diag_CHF = lambda df:df.Diagnosis.str.contains('HEART FAILURE'))\
    .assign(diag_other = lambda df: (~df.diag_hypertension)&(~df.diag_diabete)&
            (~df.diag_CAD)&(~df.diag_CPD)&(~df.diag_CHF))

In [23]:
# Get the frequency of top 5 diagnosis and other
freq_diag_fully_paid_claim = fully_paid_claim_diag\
    .groupby('id')\
    [['diag_hypertension','diag_diabete','diag_CAD','diag_CPD','diag_CHF','diag_other']].sum()\
    .rename(columns=lambda x:'freq_'+x+'_fully_paid_claim')

### Diagnosis Recency

In [24]:
# Get the recency of top 5 diagnosis and other
rec_diag_fully_paid_claim = fully_paid_claim_diag\
    .melt(['id','Days'],['diag_hypertension','diag_diabete','diag_CAD','diag_CPD','diag_CHF','diag_other'],
          'diag')\
    .query('value')\
    .groupby(['id','diag'])\
    .Days.max().abs().unstack().rename(columns=lambda x:'rec_'+x+'_fully_paid_claim')

### Monetary

In [25]:
# Get the average and total money spent for each patient
mon_full_paid_claim = HMAHCC\
    .query('event_descr == "Fully Paid Claim"')\
    .groupby('id')\
    .Member_Responsible_Amount.agg(['mean','sum'])\
    .rename(columns=lambda x:'mon_full_paid_claim_'+x)

## New diagnosis - CAD, Diabetes, Hypertension, CPD, CHF

### Frequency

In [26]:
top5_disease = ['CAD', 'Diabetes', 'Hypertension', 'CPD', 'CHF']

In [27]:
def new_diag_freq(disease):
    '''
    Calculate the frequency of frequency of new diagnosis disease
    
    :param disease: one of top5 disease
    :return: frequency of new diagnosis disease
    '''
    return HMAHCC\
        .query(f'event_descr == "New diagnosis - {disease}"')\
        .groupby('id')\
        .size().to_frame(f'new_diag_{disease}')

In [28]:
# For each of top 5 diseases, get the frequency of new diagnosis disease
results = [new_diag_freq(disease) for disease in tqdm(top5_disease)]

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 29.07it/s]


In [29]:
# Concat new diagnosis frequency for top 5 diseases
freq_new_diag = pd.concat(results, axis=1,sort=False).fillna(0)

### Recency

In [30]:
def new_diag_rec(disease):
    '''
    Calculate the recency of frequency of new diagnosis disease
    
    :param disease: one of top5 disease
    :return: recency of new diagnosis disease
    '''
    return HMAHCC\
        .query(f'event_descr == "New diagnosis - {disease}"')\
        .groupby('id')\
        .Days.max().abs().to_frame(f'rec_new_diag_{disease}')

In [31]:
# For each of top 5 diseases, get the recency of new diagnosis disease
results = [new_diag_rec(disease) for disease in tqdm(top5_disease)]

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 25.00it/s]


In [32]:
# Concat new diagnosis recency for top 5 diseases
rec_new_diag = pd.concat(results, axis=1,sort=False)

## Surgery

### Frequency surgery diagnosis

In [33]:
# Get all the words from diagnosis column, count the term frequency of each word
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(stop_words='english')
tdm = vec.fit_transform(HMAHCC.query('event_descr == "Surgery"').Diagnosis)
term_frequency = pd.Series(np.array(tdm.sum(axis=0)).reshape(-1),vec.get_feature_names())

In [34]:
# Manually select top 100 disease or human organs that may related to LTOT
top_surgery = ["malignant","neoplasm","colon","cataract","chronic","screening","lumbar",
               "eye","radiculopathy","coronary","pain","spondylosis","lumbosacral",
               "myelopathy","ulcer","artery","benign","neoplasms","breast","initial","heart",
               "atherosclerosis","angina","skin","history","pectoris","foot","intervertebral",
               "knee","atherosclerotic","sclerosis","hernia","gangrene","primary","stenosis",
               "hemorrhage","polyps","colonic","bleeding","female","kidney","leg",
               "unilateral","gastritis","cholecystitis","cervical","diabetes","prostate",
               "esophageal","acute","disorders","limb","shoulder","venous","fracture",
               "urinary","spinal","mellitus","pressure","gallbladder","neuritis","secondary",
               "osteoarthritis","radiculitis","thoracic","inguinal","recurrent","carcinoma",
               "tear","arteries","bladder","carpal","dysphagia","esophagus","reflux","hip",
               "chest","meniscus","esophagitis","senile","ankle","anemia","polyp","wound",
               "rectum","cardiac","claudication","ureter","atrial","renal","sacroiliitis",
               "gastro","fibrillation","hypertension","vascular","lung","osteomyelitis",
               "intestine","anus","rupture"]

In [35]:
# Get the index of each top 100 words in the term document matrix
word_index = [vec.get_feature_names().index(word) for word in top_surgery]

In [36]:
# Calculate the frequency of top 100 words
freq_surgery_diag = pd.DataFrame(tdm[:,word_index].todense(),columns=top_surgery)\
    .rename(columns=lambda x: 'freq_surgery_diag_'+x)\
    .set_index(HMAHCC.query('event_descr == "Surgery"').id)\
    .groupby('id')\
    .sum()

### Frequency place of treatment

In [37]:
# Calculate the frequency of place of treatment for surgery
freq_surgery_place = HMAHCC\
    .query('event_descr == "Surgery"')\
    .groupby(['id','Place_of_Treatment'])\
    .size().unstack(fill_value=0)\
    .rename(columns=lambda x:'freq_surgery_place_'+x)

### Recency place of treatment

In [38]:
# Calculate the recency of place of treatment for surgery
rec_surgery_place = HMAHCC\
    .query('event_descr == "Surgery"')\
    .groupby(['id','Place_of_Treatment'])\
    .Days.max().abs().unstack().rename(columns=lambda x:'rec_surgery_place_'+x)

## New provider

### Frequency

In [39]:
# Calculate the frequency of new provider
freq_new_provider = HMAHCC\
    .query('event_descr == "New provider"')\
    .groupby(['id'])\
    .size().to_frame('freq_new_provider')

### Recency

In [40]:
# Calculate the recency of new provider
rec_new_provider = HMAHCC\
    .query('event_descr == "New provider"')\
    .groupby(['id'])\
    .Days.max().abs().to_frame('rec_new_provider')

## RX Claim - Paid

### Frequency

In [41]:
# Calculate the frequency of each drug each patient uses
freq_rx_claim_paid = HMAHCC\
    .query('event_descr == "RX Claim - Paid"')\
    .groupby(['id','Drug_Group_Description'])\
    .size().unstack(fill_value=0)\
    .rename(columns=lambda x:'freq_paid_'+x)

### Recency

In [42]:
# Calculate the recency of each drug each patient uses
rec_rx_claim_paid = HMAHCC\
    .query('event_descr == "RX Claim - Paid"')\
    .groupby(['id','Drug_Group_Description'])\
    .Days.max().abs().unstack()\
    .rename(columns=lambda x:'rec_paid_'+x)

### Monetary

In [43]:
# Calculate the average and total money each patient spends on each drug
mon_rx_claim_paid = HMAHCC\
    .query('event_descr == "RX Claim - Paid"')\
    .groupby('id')\
    .Member_Responsible_Amount.agg(['mean','sum'])\
    .rename(columns=lambda x:'mon_paid_'+x)

### PAY_DAY_SUPPLY_CNT

In [44]:
# Calculate the average and total days supply each patient has on each drug
pay_day_supply_cnt = HMAHCC\
    .query('event_descr == "RX Claim - Paid"')\
    .groupby('id')\
    .PAY_DAY_SUPPLY_CNT.agg(['mean','sum'])\
    .rename(columns=lambda x:x+'_pay_day_supply_cnt')

### MME

#### Mean & Sum

In [45]:
# Calculate the average and total MME each patient has on each drug
MME = HMAHCC\
    .query('event_descr == "RX Claim - Paid"')\
    .assign(MME_per_rx=lambda df:df.MME*df.PAY_DAY_SUPPLY_CNT)\
    .groupby('id')\
    .MME_per_rx.agg(['mean','sum'])\
    .rename(columns=lambda x:x+'_MME_per_rx')

#### Frequency

In [46]:
# Calculate the frequency of MME drug each patient uses
freq_MME = HMAHCC\
    .query('event_descr == "RX Claim - Paid"')\
    .query('MME.notnull()',engine='python')\
    .groupby('id')\
    .size()\
    .to_frame('freq_MME')

#### Recency

In [47]:
# Calculate the recency of MME drug each patient uses
rec_MME = HMAHCC\
    .query('event_descr == "RX Claim - Paid"')\
    .query('MME.notnull()',engine='python')\
    .query('Days<0')\
    .groupby('id')\
    .Days.max().abs()\
    .to_frame('rec_MME')

### DRUG_TYPE

In [48]:
# Calculate the frequency of each drug type for each patient
drug_type = HMAHCC\
    .query('event_descr == "RX Claim - Paid"')\
    .groupby(['id','DRUG_TYPE'])\
    .size().unstack(fill_value=0)\
    .rename(columns=lambda x:'freq_drug_type_'+x)

### Specialty

In [49]:
# Calculate the frequency of each specialty for each patient
specialty = HMAHCC\
    .query('event_descr == "RX Claim - Paid"')\
    .groupby(['id','Specialty'])\
    .size().unstack(fill_value=0)\
    .rename(columns=lambda x:'freq_specialty_'+x)

## RX Claim - New Drug

### Frequency

In [50]:
# Calculate the frequency of each new drug each patient uses
freq_rx_claim_new_drug = HMAHCC\
    .query('event_descr == "RX Claim - New Drug"')\
    .groupby(['id','Drug_Group_Description'])\
    .size().unstack(fill_value=0)\
    .rename(columns=lambda x:'freq_new_'+x)

### Recency

In [51]:
# Calculate the recency of each new drug each patient uses
rec_rx_claim_new_drug = HMAHCC\
    .query('event_descr == "RX Claim - New Drug"')\
    .groupby(['id','Drug_Group_Description'])\
    .Days.max().abs().unstack()\
    .rename(columns=lambda x:'rec_new_'+x)

## RX Claim - First Time Mail Order

### Frequency

In [52]:
# Calculate the frequency of each mail order drug each patient uses
freq_rx_claim_mail = HMAHCC\
    .query('event_descr == "RX Claim - First Time Mail Order"')\
    .groupby(['id','Drug_Group_Description'])\
    .size().unstack(fill_value=0)\
    .rename(columns=lambda x:'freq_mail_'+x)

### Recency

In [53]:
# Calculate the recency of each mail order drug each patient uses
rec_rx_claim_mail = HMAHCC\
    .query('event_descr == "RX Claim - First Time Mail Order"')\
    .groupby(['id','Drug_Group_Description'])\
    .Days.max().abs().unstack()\
    .rename(columns=lambda x:'rec_mail_'+x)

## Combine Features and Target

In [54]:
# Calculate # of negative days for each of patient
nbr_negative_days = HMAHCC.groupby('id').Days.min().abs().to_frame('nbr_negative_days')

In [55]:
# Put all the features in a list
feature_list = [freq_inbound_call_by_mbr,rec_inbound_call_by_mbr,freq_inbound_call_by_other,
 rec_inbound_call_by_other,freq_inbound_call_by_prov,rec_inbound_call_by_prov,freq_diag_fully_paid_claim,
 rec_diag_fully_paid_claim,mon_full_paid_claim,freq_new_diag,rec_new_diag,freq_surgery_diag,
 freq_surgery_place,rec_surgery_place,freq_new_provider,rec_new_provider,freq_rx_claim_paid,
 rec_rx_claim_paid,mon_rx_claim_paid,pay_day_supply_cnt,MME,freq_MME,rec_MME,drug_type,specialty,
 freq_rx_claim_new_drug,rec_rx_claim_new_drug,freq_rx_claim_mail,rec_rx_claim_mail]

In [56]:
# Concat all features into one dataframe
features = pd.concat(feature_list,axis=1,sort=False)

In [57]:
# Join features with target variable, and join with # of negative days
design_matrix = pd.merge(features,nbr_negative_days,how='inner',left_index=True,right_index=True)

In [58]:
# Normalize frequency by number of days
design_matrix.loc[:,design_matrix.columns.str.contains('freq_')] = design_matrix.loc[:,design_matrix.columns.str.contains('freq_')]\
    .apply(lambda col:col/design_matrix.nbr_negative_days)
design_matrix.drop(columns=['nbr_negative_days'],inplace=True)

In [59]:
design_matrix.shape

(5998, 460)

In [60]:
# Save design matrix to parquet
design_matrix.to_parquet('design_matrix_holdout.parquet')

# Modeling

In [69]:
# Read in design matrix
design_matrix = pd.read_parquet('design_matrix_holdout.parquet')

In [62]:
#load saved model
xgb = joblib.load('xgb.model')

In [63]:
# Make the columns in hold out set the same as in training set
design_matrix = design_matrix\
    .assign(**{col:0 for col in set(xgb.get_booster().feature_names) - set(design_matrix.columns)})\
    .drop(list(set(design_matrix.columns) - set(xgb.get_booster().feature_names)),axis=1)\
    [xgb.get_booster().feature_names]

In [64]:
# Predict probability of holdout set
y_pred = xgb.predict_proba(design_matrix)

In [66]:
# Concat prediction and ID, and create rank for each patient. For ties, use min method
prediction = pd.Series(y_pred[:,1],index=design_matrix.index,name='SCORE')\
    .to_frame()\
    .reindex(HMAHCC_COMP.id.unique(),fill_value=0)\
    .assign(RANK=lambda df:df.SCORE.rank(method='min',ascending=False))\
    .reset_index()\
    .rename(columns={'index':'ID'})\
    .sort_values('RANK')

In [67]:
# Save prediction to csv
prediction.to_csv('CaseCompetition_Yueyi_Wang.csv',index=False)